In [12]:
!pip install beautifulsoup4
!pip install requests
!pip install requests_html

In [13]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import pandas as pd

ImportError: dlopen(/Users/alexsmith/Programming/Projects/ticket-predictor/.venv/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so, 0x0002): symbol not found in flat namespace (_exsltDateXpathCtxtRegister)

In [28]:
import requests

home_url = 'https://www.livefootballtickets.com/euro-cup-tickets.html'
country_url = 'https://www.livefootballtickets.com/euro-cup/albania-football-tickets.html'
match_url = 'https://www.livefootballtickets.com/fixtures/match-1-germany-v-a2-tickets-euro-2024.html'

html_text = requests.get(country_url).text

match_page_text = requests.get(match_url).text

print(match_page_text)



<!DOCTYPE html>
<html class="no-js" lang="en">
    <head><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="msvalidate.01" content="4E29BE86609662993923B68A064E1BCA" /><link href="/main/styles?v=1Awv-O0I-PHJnPjwWzyRq5r_1Zx71-_7nvtg4BTwFTM1" rel="stylesheet"/>
<script>
window.dataLayer = window.dataLayer || [];
window.dataLayer.push({
event: 'eec.detail',
ecommerce: {
detail: {
products: [{
'id':'37875','name':'MATCH 1 - Germany v Scotland','category':'Euro 2024','brand':'MATCH 1 - Germany'}] } } });
</script>
<!--Google Tag Manager -->
<script> (function(w, d, s, l, i){ w[l] = w[l] ||[]; w[l].push({'gtm.start': new Date().getTime(),event:'gtm.js'});var f = d.getElementsByTagName(s)[0], j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async=true;j.src= '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f); })
(window,document,'script','dataLayer','GTM-5F72PW');</script><!-- End Google Tag Mana

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_text, 'html.parser')

prices = soup.find_all("div", {"class": "ticket-price"})

In [52]:
import re
# import pandas as pd

soup = BeautifulSoup(requests.get(home_url).text, 'html.parser')

# Find all the links to country pages within the 'team-list' div
country_links = soup.select('div.team-list a.team-list-item')

# List to hold the URLs and the HTML content of each country page
country_pages_content = []

all_match_links = []

# Iterate through the list of country links
for link in country_links:
    country_url = link['href']
    country_name = link.get_text(strip=True)
    
    # Send a GET request to the country page
    country_response = requests.get(country_url)
    country_page_content = country_response.text

    # Store the URL and content in a dictionary, then add to the list
    country_pages_content.append({
        'country_name': country_name,
        'url': country_url,
        'html_content': country_page_content
    })

for country in country_pages_content:

    country_url = country['url']
    # Find all the 'a' elements within the 'h4' tags inside 'event-list-item-inner' divs

    country_soup = BeautifulSoup(requests.get(country_url).text, 'html.parser')   

    match_links = country_soup.select('.event-list-item-inner .event-info-name a')

    # Extract the 'href' attribute (the URL) from each link
    match_urls = [a['href'] for a in match_links]

    print(country['country_name'])

    print(match_urls)

    for match_url in match_urls:
        if match_url in all_match_links:
            pass
        else:
            match_soup = BeautifulSoup(requests.get(match_url).text, 'html.parser')
            venue = match_soup.find(attrs={"data-qa": "event-venue"}).get_text(strip=True)
            # Find the element with the data-qa attribute set to 'event-location'
            location = match_soup.find(attrs={"data-qa": "event-location"}).get_text(strip=True)
            tickets_available = match_soup.find(class_="tickets-available").span.get_text(strip=True)
            tickets = match_soup.select('li.ticket-row')

                # Find the element with the class 'box-header--title'
            title_element = match_soup.find(class_="box-header--title").get_text(strip=True)
            
            print('titile', title_element)
            title_pattern = r'MATCH (\d+) - (.+?)v(.+) Tickets'
            title_match = re.search(title_pattern, title_element)

            # Define a regex pattern to extract the team codes from the URL
            url_pattern = r'match-\d+-(\w+)-v-(\w+)-tickets'
            url_match = re.search(url_pattern, match_url)

            if title_match and url_match:
                print('here')
                match_number = title_match.group(1)
                team_one = title_match.group(2).strip()
                team_two = title_match.group(3).strip()
                team_one_code = url_match.group(1).upper()
                team_two_code = url_match.group(2).upper()
                print(match_number, team_one, team_two, team_one_code, team_two_code)


            ticket_data = []

            # print('M', match_number, home, away)

            for ticket in tickets:
                stadium_id = ticket['data-stadium-id']
                ticket_heading = ticket.find('h5', class_='ticket-heading').text
                ticket_price = ticket.find('div', class_='ticket-price').h4.text
                ticket_quantities = [int(option['value']) for option in ticket.select('select.ticket-quantity option')]
                
                ticket_dict = {
                    'stadium_id': stadium_id,
                    # 'game_id': match_number,
                    'ticket_heading': ticket_heading,
                    'ticket_price': ticket_price,
                    'ticket_quantities': ticket_quantities
                    }
                ticket_data.append(ticket_dict)
                
                print(f"Stadium ID: {stadium_id}")
                print()
                print(f"Ticket Heading: {ticket_heading}")
                print(f"Ticket Price: {ticket_price}")
                # print(f"Highest Ticket Quantity: {highest_quantity}")
                print("-----")
            # tickets_df = pd.Dataframe(ticket_data)


print(len(country_pages_content))

ConnectionError: HTTPSConnectionPool(host='www.livefootballtickets.com', port=443): Max retries exceeded with url: /euro-cup-tickets.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x114951ab0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [11]:
tickets = soup.select('li.ticket-row')

for ticket in tickets:
    stadium_id = ticket['data-stadium-id']
    ticket_heading = ticket.find('h5', class_='ticket-heading').text
    ticket_price = ticket.find('div', class_='ticket-price').h4.text
    ticket_quantities = [int(option['value']) for option in ticket.select('select.ticket-quantity option')]
    # highest_quantity = max(ticket_quantities)
    
    print(f"Stadium ID: {stadium_id}")
    print(f"Ticket Heading: {ticket_heading}")
    print(f"Ticket Price: {ticket_price}")
    # print(f"Highest Ticket Quantity: {highest_quantity}")
    print("-----")


Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 479.00
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 479.98
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.95
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.96
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.97
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 480.98
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 482.00
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 486.00
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £486.22
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket Price: £ 487.99
-----
Stadium ID: C_1950
Ticket Heading: CATEGORY 3 - NEUTRAL TICKETS
Ticket 

In [ ]:
soup = BeautifulSoup(requests.get(home_url).text, 'html.parser')
country_links = soup.select('div.team-list a.team-list-item')

